Testing CNN model

In [ ]:
import tensorflow as tf
import pandas as pd

from keras import Sequential, layers, callbacks, losses, utils, applications
from keras.preprocessing import image_dataset_from_directory




: 

In [ ]:
%%capture
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')

Preprocess data and create the training and validation split 80% / 20%.

In [ ]:
img_height = 180
img_width = 180

batch_size = 32

seed = 123

data_dir = "../data/tests/flower_photos"

#input_shape = [img_height,img_width, 3]

ds_train = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    seed=seed,
    subset="training",
    image_size=(img_height,img_width),
    batch_size=batch_size
    )
ds_valid= image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    seed=seed,
    subset="validation",
    image_size=(img_height,img_width),
    batch_size=batch_size
    )

class_names = ds_train.class_names # type: ignore
num_classes = 5
#print(class_names)

In [ ]:
%%capture
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in ds_train.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
%%capture
for image_batch, labels_batch in ds_train:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

ds_train = ds_train.cache().prefetch(buffer_size=AUTOTUNE)
ds_valid = ds_valid.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:

layers = [
    # Body
    layers.Rescaling(1./255),
    
    layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    layers.MaxPooling2D(),
    #layers.Dropout(rate=0.3),

    layers.Conv2D(filters=128, kernel_size=3,activation='relu'),
    layers.MaxPooling2D(),
    #layers.Dropout(rate=0.3),
    
    # Classifier head
    layers.Flatten(),
    layers.Dense(32, activation="relu"),
    #layers.Dropout(rate=0.2),
    layers.Dense(num_classes),
]

asdasd

In [ ]:
model = Sequential(layers=layers)

early_stopping = callbacks.EarlyStopping(patience=5,
                                      min_delta=0.001,restore_best_weights=True)
model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

Train the model

In [ ]:

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    callbacks=[early_stopping],
    epochs=40,
)

In [ ]:

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()